In [1]:
from functools import partial

from transformers import CLIPVisionModel 
import torch
from torch import nn
from torchvision import transforms
from PIL import Image

import torch
import torch.nn as nn
from transformers import CLIPVisionModel
from torchvision import transforms

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('device:', device)



device: cuda:1


In [2]:
train_pixel_img_feature = torch.load('/userhome2/liweile/EEG_Image_decode/Generation/variables/ViT-L-14_features_multimodal_train.pt', map_location=device, weights_only=True)['img_features']#
test_pixel_img_feature = torch.load('/userhome2/liweile/EEG_Image_decode/Generation/variables/ViT-L-14_features_multimodal_test.pt', map_location=device, weights_only=True)['img_features']#
train_img_feature = torch.load('/userhome2/liweile/EEG_Image_decode/Generation/variables/ViT-H-14_features_train.pt', map_location=device, weights_only=True)['img_features'].unsqueeze(1)#
test_img_feature = torch.load('/userhome2/liweile/EEG_Image_decode/Generation/variables/ViT-H-14_features_test.pt', map_location=device, weights_only=True)['img_features'].unsqueeze(1)#


In [3]:
print(train_pixel_img_feature.shape)
print(test_pixel_img_feature.shape)
print(train_img_feature.shape)
print(test_img_feature.shape)

torch.Size([16540, 256, 1024])
torch.Size([200, 256, 1024])
torch.Size([16540, 1, 1024])
torch.Size([200, 1, 1024])


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from einops.layers.torch import Rearrange, Reduce

import datetime
# Define the neural network
class PixelProjector(nn.Sequential):
    def __init__(self, proj_dim=1024):
        super().__init__(
            Rearrange('B C L->B L C'),    
            nn.Linear(1, 256),
            nn.LayerNorm(256),
            Rearrange('B L C->B C L'),
            nn.Linear(1024, 1024),
            nn.LayerNorm(proj_dim),
            )
        
        

# Instantiate the model, loss function, and optimizer

model = PixelProjector(proj_dim=1024).to(torch.bfloat16).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.1)

# Prepare data loaders
train_dataset = TensorDataset(train_img_feature, train_pixel_img_feature)
test_dataset = TensorDataset(test_img_feature, test_pixel_img_feature)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"{[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]} Prepare data loaders done.")

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"{[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]} Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Testing loop
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(torch.bfloat16).to(device), targets.to(torch.bfloat16).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

print(f"{[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]} Test Loss: {test_loss/len(test_loader)}")

# Save the trained model
torch.save(model.state_dict(), '/userhome2/liweile/EEG_Image_decode/Generation/EEG_caption/model_weights/PixelProjector_best.bin')
print(f"{[datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]} Model saved as PixelProjector.bin")


['2025-04-09 15:35:59'] Prepare data loaders done.
['2025-04-09 15:36:01'] Epoch 1/30, Loss: 0.9113977713178295
['2025-04-09 15:36:02'] Epoch 2/30, Loss: 0.7538154069767442
['2025-04-09 15:36:03'] Epoch 3/30, Loss: 0.7460483284883721
['2025-04-09 15:36:05'] Epoch 4/30, Loss: 0.7436864098837209
['2025-04-09 15:36:06'] Epoch 5/30, Loss: 0.7425660125968992
['2025-04-09 15:36:08'] Epoch 6/30, Loss: 0.741718144379845
['2025-04-09 15:36:09'] Epoch 7/30, Loss: 0.7411428052325582
['2025-04-09 15:36:10'] Epoch 8/30, Loss: 0.7405069040697675
['2025-04-09 15:36:12'] Epoch 9/30, Loss: 0.7401738129844961
['2025-04-09 15:36:13'] Epoch 10/30, Loss: 0.7397044573643411
['2025-04-09 15:36:14'] Epoch 11/30, Loss: 0.7395833333333334
['2025-04-09 15:36:16'] Epoch 12/30, Loss: 0.739265382751938
['2025-04-09 15:36:17'] Epoch 13/30, Loss: 0.7389928536821705
['2025-04-09 15:36:18'] Epoch 14/30, Loss: 0.7386900436046512
['2025-04-09 15:36:19'] Epoch 15/30, Loss: 0.7386446220930233
['2025-04-09 15:36:21'] Epoch 